In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

Format of the file: age_gender_ethnicity_datetime 

In [2]:
path = "/kaggle/input/utkface-new/UTKFace/"
files = os.listdir(path)
size = len(files)
print("Total samples:",size)
print(files[0])

In [91]:
import cv2
images = []
ages = []
genders = []
for file in files:
    image = cv2.imread(path+file,0)
    image = cv2.resize(image,dsize=(128,128))
    image = image.reshape((image.shape[0],image.shape[1],1))
    images.append(image)
    split_var = file.split('_')
    ages.append(split_var[0])
    genders.append(int(split_var[1]) )

In [92]:
# images = images[:5000]
# ages = ages[:5000]
# genders = genders[:5000]
# size = len(ages)

In [93]:
import matplotlib.pyplot as plt
x_ages = list(set(ages))
y_ages = [ages.count(i) for i in x_ages]
plt.bar(x_ages,y_ages)
plt.show()
print("Max value:",max(ages))

In [94]:
def display(img):
    plt.imshow(img[:,:,0])
    plt.set_cmap('gray')
    plt.show()
idx = 500
sample = images[idx]
print("Gender:",genders[idx],"Age:",ages[idx])
display(sample)

In [95]:
# def age_group(age):
#     if age >=0 and age < 10:
#         return 1
#     elif age < 20:
#         return 2
#     elif age < 30:
#         return 3
#     elif age < 40:
#         return 4
#     elif age < 50:
#         return 5
#     elif age < 60:
#         return 6
#     elif age < 70:
#         return 7
#     elif age < 80:
#         return 8
#     elif age < 90:
#         return 9
#     else:
#         return 10

def age_group(age):
    if age >=0 and age < 18:
        return 0
    else:
        return 1

In [96]:
# pre processing
target = np.zeros((size,2),dtype='float32')
features = np.zeros((size,sample.shape[0],sample.shape[1],1),dtype = 'float32')
for i in range(size):
    target[i,0] = age_group(int(ages[i]))
    target[i,1] = int(genders[i])
    features[i] = images[i]
features = features / 255
display(features[550])

In [97]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.1,shuffle  = True)
print("Samples in Training:",x_train.shape[0])
print("Samples in Testing:",x_test.shape[0])

In [98]:
print("Shape of image:",sample.shape)

In [99]:
import keras 
from keras.layers import *
from keras.models import *
from keras import backend as K

In [1]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def Convolution(input_tensor,filters):
    
    x = Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(input_tensor)
    x = Dropout(0.1)(x)
    x= Activation('relu')(x)
    return x

In [2]:
inputs = Input(shape=(128,128,1))
conv_1= Convolution(inputs,32)
maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)
conv_2 = Convolution(maxp_1,64)
maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)
conv_3 = Convolution(maxp_2,128)
maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)
conv_4 = Convolution(maxp_3,256)
maxp_4 = MaxPooling2D(pool_size = (2, 2)) (conv_4)
flatten= Flatten() (maxp_4)
dense_1= Dense(64,activation='selu')(flatten)
dense_2= Dense(64,activation='selu')(flatten)
gender_model = Dropout(0.2)(dense_2)
gender_model = Dense(32, activation='selu')(gender_model)
gender_model = Dropout(0.2)(gender_model)
gender_model = Dense(16, activation='selu')(gender_model)
drop_1=Dropout(0.2)(dense_1)
drop_2=Dropout(0.2)(gender_model)
output_1= Dense(1,activation="sigmoid",name='sex_out')(drop_1)
output_2= Dense(1,activation="sigmoid",name='age_out')(drop_2)
model = Model(inputs=[inputs], outputs=[output_1,output_2])
model.compile(loss=["binary_crossentropy","binary_crossentropy"], optimizer="Adam",metrics=["accuracy"])

In [3]:
model.summary()

In [103]:
h = model.fit(x_train,[y_train[:,0],y_train[:,1]],validation_split=0.15,epochs = 30, batch_size=128,shuffle = True)

In [16]:
model.save('data.h5')

In [106]:
history = h
plt.plot(history.history['age_out_accuracy'])
plt.plot(history.history['val_age_out_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [107]:
history = h
plt.plot(history.history['sex_out_accuracy'])
plt.plot(history.history['val_sex_out_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [108]:
def display(img):
    plt.imshow(img[:,:,0])
    plt.set_cmap('gray')
    plt.show()
    
# def age_group(age):
#     if age >=0 and age < 10:
#         return 1
#     elif age < 20:
#         return 2
#     elif age < 30:
#         return 3
#     elif age < 40:
#         return 4
#     elif age < 50:
#         return 5
#     elif age < 60:
#         return 6
#     elif age < 70:
#         return 7
#     elif age < 80:
#         return 8
#     elif age < 90:
#         return 9
#     else:
#         return 10

# def get_age(distr):
#     distr = distr*10
#     if distr >= 0 and distr <= 1 :return "0-10"
#     if distr >= 1 and distr <= 2 :return "10-20"
#     if distr >= 2 and distr <= 3 :return "20-30"
#     if distr >= 3 and distr <= 4 :return "30-40"
#     if distr >= 4 and distr <= 5 :return "40-50"
#     if distr >= 5 and distr <= 6:return "50-60"
#     if distr >= 6 and distr <= 7:return "60-70"
#     if distr >= 7 and distr <= 8:return "70-80"
#     if distr >= 8 and distr <= 9 :return "80-90"
#     if distr >= 9 and distr <= 10 :return "90-100"
#     return "Unknown"

def get_age(distr):
    if distr < 0.5:return "child, age is 0 -18"
    else: return "Adult, age is 18+"
    
def get_gender(prob):
    if prob < 0.5:return "Male"
    else: return "Female"

def get_result(sample):
    sample = sample/255
    val = model.predict( np.array([ sample ]) )    
    age = get_age(val[0])
    gender = get_gender(val[1])
    print("Values:",val,"\nPredicted Gender:",gender,"Predicted Age:",age)
    
    
indexes = [500,59,80,2,4546,7,9,256,45]
for idx in indexes:
    sample = images[idx]
    display(sample)
    print("Actual Gender:",get_gender(genders[idx]),"Age:",ages[idx])
    res = get_result(sample)

In [ ]:
from sklearn.metrics import confusion_matrix
test_output = model.predict(test_input, verbose=0)
print(confusion_matrix(Y_test, test_output))

In [51]:
# import cv2
# ages=[]
# genders=[]
# images=[]
# for fle in files:
#   age=int(fle.split('_')[0])
#   gender=int(fle.split('_')[1])
#   total=fldr+'/'+fle
#   print(total)
#   image=cv2.imread(total)
#   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#   image= cv2.resize(image,(48,48))
#   images.append(image)
#   ages.append(age)
#   genders.append(gender)

In [52]:
import cv2
images = []
ages = []
genders = []
for file in files:
    image = cv2.imread(path+file,0)
    image = cv2.resize(image,dsize=(48,48))
    image = image.reshape((image.shape[0],image.shape[1],1))
    images.append(image)
    split_var = file.split('_')
    ages.append(split_var[0])
    genders.append(int(split_var[1]) )

In [15]:
labels=[]
i=0
while i<len(ages):
    label=[]
    label.append([ages[i]])
    label.append([genders[i]])
    labels.append(label)
    i+=1

In [16]:
images_f=np.array(images)
labels_f=np.array(labels)
images_f_2=images_f/255
X_train, X_test, Y_train, Y_test= train_test_split(images_f_2, labels_f,test_size=0.25)

In [55]:
Y_train_2=[Y_train[:,1],Y_train[:,0]]
Y_test_2=[Y_test[:,1],Y_test[:,0]]

In [56]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def Convolution(input_tensor,filters):
    
    x = Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same',strides=(1, 1),kernel_regularizer=l2(0.001))(input_tensor)
    x = Dropout(0.1)(x)
    x= Activation('relu')(x)
    return x

In [57]:
def model(input_shape):
    inputs = Input((input_shape))
    conv_1= Convolution(inputs,32)
    maxp_1 = MaxPooling2D(pool_size = (2,2)) (conv_1)
    conv_2 = Convolution(maxp_1,64)
    maxp_2 = MaxPooling2D(pool_size = (2, 2)) (conv_2)
    conv_3 = Convolution(maxp_2,128)
    maxp_3 = MaxPooling2D(pool_size = (2, 2)) (conv_3)
    conv_4 = Convolution(maxp_3,256)
    maxp_4 = MaxPooling2D(pool_size = (2, 2)) (conv_4)
    flatten= Flatten() (maxp_4)
    dense_1= Dense(64,activation='relu')(flatten)
    dense_2= Dense(64,activation='relu')(flatten)
    drop_1=Dropout(0.2)(dense_1)
    drop_2=Dropout(0.2)(dense_2)
    output_1= Dense(1,activation="sigmoid",name='sex_out')(drop_1)
    output_2= Dense(1,activation="relu",name='age_out')(drop_2)
    model = Model(inputs=[inputs], outputs=[output_1,output_2])
    model.compile(loss=["binary_crossentropy","mae"], optimizer="Adam",metrics=["accuracy"])
    return model